# Import Libraries

In [1]:
#import all required libraries
from decimal import Decimal
from io import BytesIO
import json
import logging
import os
import requests
from zipfile import ZipFile
from boto3.dynamodb.conditions import Key
from boto3 import Session
from botocore.exceptions import ClientError
from configparser import ConfigParser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import time
import csv
from datetime import date
import pandas as pd
import re


from src.beer_functions import Beers
from src.utils import (setup_driver,
                       authenticate_age,
                       extract_text)

In [2]:
driver = setup_driver()

In [3]:
session = Session(profile_name='ds1')
dyn_resource = session.resource("dynamodb")
table = dyn_resource.Table('md-brewery')

In [51]:
# table = dyn_resource.create_table(TableName = 'md-brewery',
#                               KeySchema=[
#                     {"AttributeName": "scraping_date", "KeyType": "HASH"},  # Partition key
#                     {"AttributeName": "beer_name", "KeyType": "RANGE"},  # Sort key
#                 ],
#                 AttributeDefinitions=[
#                     {"AttributeName": "scraping_date", "AttributeType": "S"},
#                     {"AttributeName": "beer_name", "AttributeType": "S"},
#                 ],
#                 ProvisionedThroughput={
#                     "ReadCapacityUnits": 5,
#                     "WriteCapacityUnits": 5,
#                 },)
# table.wait_until_exists()

In [4]:
with open("beer_xpaths.json") as file:
    beer_json: dict = json.load(file)
with open("site_access.json") as file:
    access_dict: dict = json.load(file)

## Functions

In [5]:
class Brewery:
    def __init__(self,site_info: dict,beer_info: dict):
        self.site_info = site_info
        self.beer_info = beer_info
    def __str__(self):
        return self.site_info["url"]
    def __repr__(self):
        return f"Brewery({self.site_info['url']})"
def clean_abv(raw_abv: str)-> str:
    abv_final = re.match(pattern = "[0-9]+[.]*[0-9]*",
                               string = raw_abv).group(0)
    return abv_final

## First Brewery

In [6]:
cushwa = Brewery(site_info = access_dict['cushwa'],
                 beer_info = beer_json["cushwa"])
driver.get(cushwa.site_info["url"])
authenticate_age(driver,cushwa.site_info["authentication"])

<p>I want a function that assigns to a class, checks authentication, all of that</p>

In [7]:
beer_dict = dict()
for key in cushwa.beer_info.keys():
    if key.lower() != 'abv':
        beer_dict[key] = extract_text(driver, cushwa.beer_info[key])
    else:
        beer_dict[key] = [clean_abv(abv) \
                          for abv in extract_text(driver, cushwa.beer_info[key])]

beer_dict["scraping_date"] = date.today().strftime("%m/%d/%Y")

In [8]:
pd.DataFrame(beer_dict).head(8)

,beers,style,description,abv,brewery,scraping_date
0,X Marks the Spot,Lager - Mexican,This Mexican Style Amber pours a beautiful lig...,5.4,Cushwa Brewing Co.,05/21/2024
1,Mai the 4th Be With You,Maibock,"Maibock, a pale, malt driven lager. With notes...",6.7,Cushwa Brewing Co.,05/21/2024
2,Rad Beer,Lager - American Light,When you have a pizza as good as the World Cha...,4.8,Cushwa Brewing Co.,05/21/2024
3,That'll Play,Lager - American Light,"Sky’s out limes out. That’ll Play is a clean, ...",4.2,Cushwa Brewing Co.,05/21/2024
4,That’ll Play Masters Edition,Lager - American Light,We took our That’ll Play lager and gave it the...,4.2,Cushwa Brewing Co.,05/21/2024
5,Kolschwa,Kölsch,Brewed traditionally to the German style,5,Cushwa Brewing Co.,05/21/2024
6,Pura Pils,Pilsner - Other,"An all-day crusher, packed full of flavor that...",5.5,Cushwa Brewing Co.,05/21/2024
7,Going Bridge,Red Ale - Irish,"Predominantly Irish malt, and is fermented wit...",4.8,Cushwa Brewing Co.,05/21/2024


## Write each item to dynamodb table

In [9]:
with table.batch_writer() as writer:
    for num in range(len(beer_dict['beers'])):
        writer.put_item(Item={
        "scraping_date": beer_dict["scraping_date"],
        "brewery": beer_dict["brewery"][num],
        "beer_name": beer_dict["beers"][num],
        "style": beer_dict["style"][num],
        "abv": Decimal(beer_dict["abv"][num]),
        "description": beer_dict["description"][num]
        }
        )


In [64]:
# for num in range(len(beer_dict['name'])):
#     table.put_item(
#                 Item={
#                     "scraping_date": beer_dict["scraping_date"],
#                     "brewery": beer_dict["maker"][num],
#                     "beer_name": beer_dict["name"][num],
#                     "style": beer_dict["style"][num],
#                     "abv": Decimal(str(beer_dict["abv"][num])),
#                     "description": beer_dict["description"][num]
#                     }
#             )

In [10]:
driver.quit()

# Elder Pine

In [29]:
elderpine = Brewery(beer_info=beer_json["elderpine"],
                    site_info = access_dict["elderpine"])
elderpine.beers = [col.text for col in driver.find_elements(By.XPATH,"//div[@class='sqs-html-content']/p[contains(@style,'text')]/strong")
                   if re.search("[a-zA-Z]+", col.text) and not col.text.lower().startswith('coming soon')]
styles = [col.text for col in driver.find_elements(By.XPATH,r"//html//div[@class='row sqs-row']//p[@style='text-align:center;white-space:pre-wrap;']//em")]
[col.text for col in driver.find_elements(By.XPATH,"//html//div[@class='row sqs-row']//p[@style='text-align:center;white-space:pre-wrap;']")]
grouped_attr = [col.text for col in driver.find_elements(By.XPATH,"//div[@class='sqs-html-content']//p[@style='text-align:center;white-space:pre-wrap;']//em")]

In [32]:
grouped_attr

['Oak-aged Munich-style Dark Lager ',
 '5.2% ABV',
 'Mexican-style Lager',
 '4.7% ABV',
 'Barrel-aged Dark Sour Ale',
 '9% ABV',
 'Barrel-aged Sour Citrus Ale',
 '7% ABV',
 'CAN POUR',
 'Fruited Belgian-style Witbier',
 '5% ABV',
 'West Coast-style Double IPA',
 '8.7% ABV',
 'Classic Belgian-style Witbier',
 '5% ABV',
 'CAN POUR',
 'Hazy Session IPA',
 '4.4% ABV',
 'CAN POUR',
 'Coffee Dark Mild',
 '3.8% ABV',
 'Barrel-aged Boysenberry Sour Ale',
 '8% ABV',
 'Hazy Double IPA',
 '8.4% ABV',
 'Inside Bar Only',
 'Norwegian-inspired Helles Bock Lager',
 '8% ABV',
 'CAN POUR',
 'Hazy IPA',
 '7% ABV',
 'Inside Bar Only',
 'Barrel-aged Ryewine',
 '13% ABV',
 'Inside Bar Only',
 ' Barrel-aged English-style Barleywine',
 '15.7% ABV',
 'Inside Bar Only',
 'English-style ESB',
 '5.5% ABV',
 'Inside Bar Only',
 'English-style Nut Brown Ale',
 '6.8% ABV',
 'COMING SOON',
 'COMING SOON']

# Removing the first beer in the list

## The page is using different tags for the first beer abv and style

In [26]:
# Only getting the 2nd beer onwards
grouped_attr = [col.text for col in driver.find_elements(By.XPATH,"//div[@class='sqs-html-content']//p[@style='text-align:center;white-space:pre-wrap;']//em")]

In [41]:
elderpine.abv = [clean_abv(row) for row in grouped_attr if row.__contains__(" ABV")]
elderpine.style = [row.strip() for row in grouped_attr if re.findall("^[a-zA-Z]+",row)
                   and row.lower() != "inside bar only"]


In [43]:
elderpine.style

['Oak-aged Munich-style Dark Lager',
 'Mexican-style Lager',
 'Barrel-aged Dark Sour Ale',
 'Barrel-aged Sour Citrus Ale',
 'CAN POUR',
 'Fruited Belgian-style Witbier',
 'West Coast-style Double IPA',
 'Classic Belgian-style Witbier',
 'CAN POUR',
 'Hazy Session IPA',
 'CAN POUR',
 'Coffee Dark Mild',
 'Barrel-aged Boysenberry Sour Ale',
 'Hazy Double IPA',
 'Norwegian-inspired Helles Bock Lager',
 'CAN POUR',
 'Hazy IPA',
 'Barrel-aged Ryewine',
 'English-style ESB',
 'English-style Nut Brown Ale',
 'COMING SOON',
 'COMING SOON']

# Other Half Brewing

In [25]:
beer_json["otherhalf"]

{'beers': {'xpath': False},
 'style': {'xpath': False},
 'abv': {'xpath': False},
 'description': {'xpath': False}}

In [26]:
otherhalf = Brewery(site_info = access_dict['otherhalf'],
                    beer_info = beer_json["otherhalf"])
driver.get(otherhalf.site_info["url"])
authenticate_age(driver,otherhalf.site_info["authentication"])

In [25]:
driver.quit()

# Falling Branch Brewing